# Test multi instance and multi user session

In [ ]:
!lamin login testuser1
!lamin init --storage "./testsetup-prepare"
!lamin disconnect

In [ ]:
import lamindb_setup as ln_setup
import lamindb as ln
import pytest

If you try to use lamindb, it will raise an `CurrentInstanceNotConfigured` and ask you to `init` or `connect` an instance via the python API.

In [ ]:
with pytest.raises(ln.setup.errors.CurrentInstanceNotConfigured):
    ln.track()

In [ ]:
ln_setup.core.django.reset_django()  # django is not configured, so no effect

In [ ]:
assert ln_setup.settings.user.handle == "testuser1"

In [ ]:
ln_setup.init(storage="./testsetup")

In [ ]:
assert ln_setup.settings.instance.slug == "testuser1/testsetup"

In [ ]:
from lamindb.models import User

In [ ]:
User.objects.get(handle="testuser1")

In [ ]:
with pytest.raises(Exception):  # does not exist
    User.objects.get(handle="testuser2")

In [ ]:
ln.track()

Let us try connecting to another instance:

In [ ]:
with pytest.raises(ln.setup.errors.CannotSwitchDefaultInstance) as error:
    ln.connect("testsetup3")
assert error.exconly().endswith(
    "Cannot switch default instance while `ln.track()` is live: call `ln.finish()`"
)

Let us try to init another instance in the same Python session.

In [ ]:
with pytest.raises(ln.setup.errors.CannotSwitchDefaultInstance) as error:
    ln.setup.init(storage="./testsetup2")
assert error.exconly().endswith(
    "Cannot switch default instance while `ln.track()` is live: call `ln.finish()`"
)

Switch off `ln.track()`.

In [ ]:
ln.context._transform = None

Let us login with another user:

In [ ]:
ln_setup.login("testuser2")

In [ ]:
User.objects.get(handle="testuser2")

Connect to another instance in the same process:

In [ ]:
ln_setup.connect("testuser1/testsetup-prepare")

In [ ]:
assert ln_setup.settings.instance.slug == "testuser1/testsetup-prepare"

In [ ]:
!lamin login testuser1
!lamin delete --force testsetup-prepare
!lamin delete --force testsetup